# Perspectives on matrix multiplication


Lots of you  seem to have learned how to multiply matrices ([matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication)) in high school.  
We compute the product $C=AB$ of an $m \times n$ matrix $A$ with an $n \times p$ matrix $B$ to produce an $m \times p$ matrix $C$.

Did you ever wonder why "matmul" has such a fancy definition?

When we add matrices we add elements.  Why coudn't matmul be just as easy?

## Compare Elementwise Multiply

Of course the elementwise multiply is doable but never seems to be quite as important:

(I'll bet your high school teacher never mentioned elementwise multiply!)


In [ ]:
A=[1 2
   3 4]
B=[1 2
   3 4]
@show(A.*B)    # Elementwise times is the "dot star"
@show(A*B);    # Matmul is just the "star"

For square n x n matrices, elementwise multiply requires $n^2$ operations, while matmul requires about $2n^3$. (Think $n^2$ dot products, each requiring $n$ mults and almost $n$ adds.)

## Raising the Abstraction

Why is matmul defined this way?  We will find out later in the course when we begin to understand that a matrix represents a linear transformation, and matmul is the natural representation of the composition of transformations.  It is only then you can understand the true nature of matrix multiplication.  (Bet your high school teacher never told you that!)

One of our goals in 18.06 is to sometimes stop thinking of matrices as arrays of numbers, and more as wholistic objects.

Abstractly, the rules for matrix multiplication are determined once you define how to multiply matrices by vectors $Ax$, the central [linear operation](https://en.wikipedia.org/wiki/Linear_map) of 18.06, by requiring that multiplication be [associative](https://en.wikipedia.org/wiki/Associative_property).  That is, we require:
$$
A(Bx)=(AB)x
$$
for all matrices $A$ and $B$ and all vectors $x$.  The expression $A(Bx)$ involves only matrix × vector (computing $y=Bx$ then $Ay$), and requiring this to equal $(AB)x$ actually uniquely defines the matrix–matrix product $AB$.

## Perspective 1 (high school!): rows × columns

The  most familar definition is that you take **dot products of rows of A with columns of B** to get the product $C$.  For example:
$$
\begin{pmatrix}
 -14 &   5 & 10 \\
  \color{red}{-5} & -20 & 10 \\
  -6 &  10 &  6
\end{pmatrix} =
\begin{pmatrix}
 2 & -1 & 5 \\
  \color{red}{3} &  \color{red}{4} & \color{red}{4} \\
 -4 & -2 & 0
\end{pmatrix}
\begin{pmatrix}
\color{red}{1}  & 0 & -2 \\
  \color{red}{1} & -5 &  1 \\
 \color{red}{-3} &  0 &  3
\end{pmatrix}
$$
where we have highlighted the entry $\color{red}{-5 = 3 \times 1 + 4 \times 1 + 4 \times -3}$ (second row of $A$ ⋅ first column of $B$).

This can be written out as the formula
$$
c_{ij} = \sum_{k=1}^n a_{ik} b_{kj}
$$
in terms of the entries of the matrices, e.g. $c_{ij}$ is the entry in row $i$, column $j$ of $C$, assuming $A$ has $n$ columns and $B$ has $n$ rows.

Essentially all matrix multiplications in practice are done with a version of this formula — at least, with the same operations, but often the *order* in which you multiply/add individual numbers is re-arranged.

**In this notebook, we will explore several ways to *think* about these operations by re-arranging their order.**

In [ ]:
A = [ 2  -1  5
      3   4  4
     -4  -2  0]
B = [ 1   0  -2
      1  -5   1
     -3   0   3]
C = A * B

In [ ]:
## You can write your own little program if you want to be sure you understand the algorithm:

function my_own_matmul(A,B)
   m,n1 = size(A)
   n2,p = size(B)
   if n1 ≠ n2 error("No good, n1=$(n1) ≠ n2=$(n2)") end
    
   C = [  A[i,:] ⋅ B[:,j] for i=1:m, j=1:p ]  # Matrix of dot products (explained below)
       
end
        

In [ ]:
my_own_matmul(A,B)

In [ ]:
my_own_matmul( rand(3,3), rand(2,3))


Because matrix multiplication is generally [not commutative](https://en.wikipedia.org/wiki/Commutative_property), $AB$ and $BA$ give *different* matrices:

In [ ]:
A*B - B*A

Though sometimes it can happen to be commutative.

In [ ]:
A*(A^2 + 2*A + inv(A)*10) 

In [ ]:
(A^2 + 2*A + inv(A)*10)  * A

If we want, we can compute the individual dot products in Julia too.   For example, let's compute $c_{2,1} = -5$ (the 2nd row and first column of $C$, or `C[2,1]` in Julia) by taking the dot product of the second row of $A$ with the first column of $B$.

To extract rows and columns of a matrix, Julia supports a syntax for "array slicing" pioneered by APL.  The second row of $A$ is `A[2,:]`, and the first column of `B` is `B[:,1]`:

In [ ]:
A

In [ ]:
A[2,:] # 2nd row of A

In [ ]:
B

In [ ]:
B[:,1] # 1st column of B

Now we can compute $c_{2,1}$ by their dot product via the `dot` function:

In [ ]:
dot(A[2,:], B[:,1])

In [ ]:
A[2,:] ⋅ B[:,1]   # type \cdot + tab

This matches $c_{2,1}$ from above, or `C[2,1]` in Julia:

In [ ]:
C[2,1]

In [ ]:
A[2,:]' * B[:,1]  # yet another way to get a dot product

## The summation $$c_{ij} = \sum_{k=1}^n a_{ik} b_{kj}$$ directly in a triple loop code

In [ ]:
function matmul_ijk0(A,B)
   m,n = size(A)
   n2,p = size(B)
   if n≠n2 error("No good, n=$n ≠ n2=$(n2)") end
   
   C = fill(0,m,p) # m x p "zeros" matrix
    
   for i=1:m
     for j=1:p
        for k=1:n
          C[i,j] = C[i,j] + A[i,k]*B[k,j] 
            end
        end
    end
    return C  
end
      

In [ ]:
matmul_ijk0(A,B)

## You like all those indices i,j,k, (I don't always), but you hate those three "for" loops?

In [ ]:
function matmul_ijk(A,B)
   m,n = size(A)
   n2,p = size(B)
   if n≠n2 error("No good, n=$n ≠ n2=$(n2)") end
   
   C = fill(0,m,p) # m x p "zeros" matrix
    
   for i=1:m, j=1:p, k=1:n
          C[i,j] += A[i,k]*B[k,j]   # shorthand for C[i,j] = C[i,j] + A[i,k]*B[k,j] 
   end
    
   return C  
end

In [ ]:
matmul_ijk(A,B)

## Perspective 2: matrix × columns

$AB$ can be viewed as multiplying $A$ on the *left* by each *column* of $B$.

For example, let's multiply $A$ by the first column of $B$:

In [ ]:
A*B

In [ ]:
using Interact

In [ ]:
@manipulate for j=1:3
    A * B[:,j]
end

This is the first column of $C$!  If we do this to *all* the columns of $B$, we get $C$:

In [ ]:
[ A*B[:,1]  A*B[:,2]  A*B[:,3] ] == A*B

In [ ]:
## we could make a vector of vectors to emphasize the column view
[A*B[:,j] for j=1:3]

In [ ]:
A*B

In [ ]:
hcat([A*B[:,j] for j=1:3]...)  # One can convert a vector of vectors to a matrix

Equivalently, each column of $B$ specifies a [linear combination](https://en.wikipedia.org/wiki/Linear_combination) of *columns* of $A$ to produce the columns of $C$.   So, **if you want to rearrange the *columns* of a matrix, multiply it by another matrix on the *right***.

For example, let's do the transformation that *flips the sign of the first column of $A$* and *swaps the second and third columns*.

In [ ]:
A

In [ ]:
A * [ -1  0  0
       0  0  1
       0  1  0  ]

As another example, let's swap the first two columns:

In [ ]:
A * [ 0 1 0
      1 0 0
      0 0 0 ]

In [ ]:
A

In [ ]:
function matmul_jik(A,B)
   m,n = size(A)
   n2,p = size(B)
   if n≠n2 error("No good, n=$n ≠ n2=$(n2)") end
   
   C = fill(0,m,p) # m x p "zeros" matrix
    
   for j=1:p, i=1:m, k=1:n
          C[i,j] += A[i,k]*B[k,j]   # shorthand for C[i,j] = C[i,j] + A[i,k]*B[k,j] 
   end
    
  ## recognize that the i,k loop above is really just a matrix times vector
  ## for j=1:p
  ##   C[:,j] = A * B[:,j]
  ## end
    
   return C  
end

A lot of students are perplexed.  They wonder how it could be legal to reorder in this way. 
It might take working through a few examples by hand to realize that from the perspective
of C[i,j], the same sum is accumulated in the same order, but the order in which the different elements of C finish may vary. This little Julia demo may help with this understanding.

In [ ]:
function matmul_ijk(a,b,stop)
    step=0
    n=size(a,1)
    c=zeros(a)
    for i=1:n, j=1:n, k=1:n  
        if step==stop;  return(c); end
        c[i,j] +=  a[i,k] * b[k,j]
        step+=1
    end
    c
end

function matmul_jik(a,b,stop)
    step=0
    n=size(a,1)
    c=zeros(a)
    for j=1:n, i=1:n, k=1:n  
        if step==stop;  return(c); end
        c[i,j] +=  a[i,k] * b[k,j]
        step+=1
    end
    c
end

function matmul_ikj(a,b,stop)
    step=0
    n=size(a,1)
    c=zeros(a)
    for i=1:n, k=1:n, j=1:n  
        if step==stop;  return(c); end
        c[i,j] +=  a[i,k] * b[k,j]
        step+=1
    end
    c
end

function matmul_kij(a,b,stop)
    step=0
    n=size(a,1)
    c=zeros(a)
    for k=1:n, i=1:n, j=1:n  
        if step==stop;  return(c); end
        c[i,j] +=  a[i,k] * b[k,j]
        step+=1
    end
    c
end

function matmul_jki(a,b,stop)
    step=0
    n=size(a,1)
    c=zeros(a)
    for j=1:n, k=1:n, i=1:n  
        if step==stop;  return(c); end
        c[i,j] +=  a[i,k] * b[k,j]
        step+=1
    end
    c
end

function matmul_kji(a,b,stop)
    step=0
    n=size(a,1)
    c=zeros(a)
    for k=1:n, j=1:n, i=1:n  
        if step==stop;  return(c); end
        c[i,j] +=  a[i,k] * b[k,j]
        step+=1
    end
    c
end

In [ ]:
using Interact

In [ ]:
o * o

In [ ]:
n=5
o=fill(1,n,n)
@manipulate for stop=0:n^3
    matmul_ijk(o,o,stop)
end

In [ ]:
## Perhaps a more fair matmul???
n=5
o=fill(1,n,n)
@manipulate for stop=0:n^3
    matmul_kij(o,o,stop)
end

In [ ]:
n=5
o=fill(1,n,n)
@manipulate for stop=0:n^3
    matmul_jik(o,o,stop)
end

# more wholistic matrix times column view

In [ ]:
function matmul_jik_whole(A,B,stop)
    step=0
    n=size(A,1)
    C=zeros(A)
    for j=1:n
        if step==stop;  return(C); end
        C[:,j] +=  A * B[:,j]
        step+=1
    end
    C
end

In [ ]:
n=5
o=fill(1,n,n)
@manipulate for stop=0:n
    matmul_jik_whole(o,o,stop)
end

## Perspective 3: rows × matrix

$AB$ can be viewed as multiplying each *row* of $A$ by the matrix $B$ on the *right*.  Multiplying a [row vector](https://en.wikipedia.org/wiki/Row_and_column_vectors) by a matrix on the right produces another row vector.

For example, here is the first row of $A$:

In [ ]:
A[1,:]

Whoops, slicing a matrix in Julia produces a 1d array, which is interpreted as a column vector, no matter how you slice it.  We can't multiply a column vector by a matrix $B$ on the *right* — that operation is not defined in linear algebra (the dimensions don't match up).  Julia will give an error if we try it:

In [ ]:
A

In [ ]:
A[1,:] * B

To get a row vector we must [transpose](https://en.wikipedia.org/wiki/Transpose) it.  In linear algebra, the transpose of a vector $x$ is usually denoted $x^T$.   In Julia, the transpose is `x.'`.

If we omit the `.` and just write `x'` it is the [complex-conjugate of the transpose](https://en.wikipedia.org/wiki/Conjugate_transpose), sometimes called the *adjoint*, often denoted $x^H$ (in matrix textbooks), $x^*$ (in pure math), or $x^\dagger$ (in physics).  For real-valued vectors (no complex numbers), the conjugate transpose is the same as the transpose, and correspondingly we usually just do `x'` for real vectors.

In [ ]:
A[1,:]'

Now, let's multiply this by $B$, which should give the first *row* of $C$:

In [ ]:
A[1,:]' * B

Yup!

Note that if we multiply a row vector by a matrix on the *left*, it doesn't really make sense.  Julia will give an error:

In [ ]:
B * A[1,:]'

If we multiply $B$ on the right by *all* the rows of $A$, we get $C$ again:

In [ ]:
[ A[1,:]'*B 
  A[2,:]'*B
  A[3,:]'*B ] == C

Equivalently, each row of $A$ specifies a linear combination of *rows* of $B$ to produce the rows of $C$.   So, **if you want to rearrange the *rows* of a matrix, multiply it by another matrix on the *left***.

For example, let's do the transformation that *adds two times the first row of $B$ to the third row, and leaves the other rows untouched*.  This is one of the steps of Gaussian elimination!

In [ ]:
[ 1 0 0
  -1 1 0
  3 0 1 ] * B

## Perspective 4: columns × rows

The key to this perspective is to observe:

* elements in column $i$ of $A$ only multiply elements in row $j$ of $B$
* a column times a row vector, sometimes denoted $xy^T$, is an [outer product](https://en.wikipedia.org/wiki/Outer_product) and produces a "rank-1" *matrix*

(See [this excellent paper by Gil Strang](http://mth1007.mathappl.polymtl.ca/MultFactMatrStrang.pdf) for more on this perspective applied to linear algebra. You will be in a better position to understand this at the end of 18.06, however.)

For example, here is column 1 of $A$ times row 1 of $B$:

In [ ]:
A[:,1] * B[1,:]'

If we do this for all three rows and columns and add them up, we get $C$:

In [ ]:
A[:,1] * B[1,:]' + A[:,2] * B[2,:]' + A[:,3] * B[3,:]' == C

So, from this perspective, we could write:

$$
AB = \sum_{k=1}^3 (\mbox{column } k \mbox{ of } A) (\mbox{row } k \mbox{ of } B) = \sum_{k=1}^3 A[:,k] \, B[k,:]^T
$$

where in the last expression we have used Julia notation for slices.

## Perspective 5: submatrix blocks × blocks

It turns out that all of the above are special cases of a more general rule, by which we can break up a matrix in to "submatrix" blocks and multiply the blocks.  Rows, columns, etc. are just blocks of different shapes.